In [74]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
import pandas as pd

kkma = Kkma()
okt = Okt()

In [75]:
df = pd.read_csv('Dataset.csv', sep='\t', nrows=1000)
# df

In [76]:
# s = df['content'][0]

In [77]:
# print('OKT 형태소 분석 :',okt.morphs(s))
# print('OKT 품사 태깅 :',okt.pos(s))
# print('OKT 명사 추출 :',okt.nouns(s))

In [78]:
# print('꼬꼬마 형태소 분석 :',kkma.morphs(s))
# print('꼬꼬마 품사 태깅 :',kkma.pos(s))
# print('꼬꼬마 명사 추출 :',kkma.nouns(s))  

In [79]:
# df['filtered_morphs'] = df['content'].apply(lambda x: [morph for morph, pos in okt.pos(x) if pos not in ['Josa', 'Number', 'Punctuation', 'Suffix']])
# df

In [80]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [81]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

In [82]:
X = df['content']
y = df['lable']

In [83]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='')
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [84]:
X = X.apply(lambda x: clean(x))

In [85]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')

typos_corrector = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    framework="pt",
)

X = X.apply(lambda x: typos_corrector("맞춤법을 고쳐주세요: " + x,
            max_length=128,
            num_beams=5,
            early_stopping=True)[0]['generated_text'])

In [86]:
# X

In [87]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

In [88]:
from sklearn.model_selection import train_test_split

X_list = X.values.tolist()
y = y.values
sequences = tokenizer(X_list, padding=True, truncation=True, return_tensors="np")

X_train, X_test, y_train, y_test = train_test_split(sequences['input_ids'], y, test_size=0.2, random_state=42)
X_train_mask, X_test_mask, _, _ = train_test_split(sequences['attention_mask'], y, test_size=0.2, random_state=42)

In [89]:
# sequences

In [90]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
X_train_mask_tensor = torch.tensor(X_train_mask, dtype=torch.long)
X_test_mask_tensor = torch.tensor(X_test_mask, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, X_train_mask_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, X_test_mask_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=2)

# for param in model.parameters():
#     param.requires_grad = False

# for param in model.classifier.parameters():
#     param.requires_grad = True

# 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    val_accs = []
    for input_ids, attention_mask, labels in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_losses.append(outputs.loss.item())
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            acc = (preds == labels).float().mean().item()
            val_accs.append(acc)

    val_loss = np.mean(val_losses)
    val_acc = np.mean(val_accs)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Validation Loss: 0.40685916405457717, Validation Accuracy: 0.8317307692307693
Epoch 2/10, Validation Loss: 0.3859073084134322, Validation Accuracy: 0.8269230769230769
Epoch 3/10, Validation Loss: 0.39045279673658884, Validation Accuracy: 0.8557692307692307
Epoch 4/10, Validation Loss: 0.47200964534511936, Validation Accuracy: 0.8365384615384616
Epoch 5/10, Validation Loss: 0.5625276542626895, Validation Accuracy: 0.8413461538461539
Epoch 6/10, Validation Loss: 0.5712079332711605, Validation Accuracy: 0.8653846153846154
Epoch 7/10, Validation Loss: 0.6083593733178881, Validation Accuracy: 0.8701923076923077
Epoch 8/10, Validation Loss: 0.6923117442056537, Validation Accuracy: 0.8173076923076923
Epoch 9/10, Validation Loss: 0.5330127471914659, Validation Accuracy: 0.8701923076923077
Epoch 10/10, Validation Loss: 0.5876648701154269, Validation Accuracy: 0.8413461538461539
